## 1. 사용할 패키지 불러오기

In [76]:
import pandas as pd
from glob import glob
import numpy as np
import re

## 2. 데이터 불러오기

In [77]:
xlsx_list = glob("./data/table_data/*.xlsx")
data = pd.read_excel(xlsx_list[0])
for xlsx_file in xlsx_list[1:]:
    data = pd.concat([data, pd.read_excel(xlsx_file)], axis=0).reset_index(drop = True)

data = data.iloc[:, 1:10]
data = data.loc[-data['이미지 고유 번호'].isna(), :].reset_index(drop = True)
data.head()

,작가명,제목,재료,사이즈,제작년도,판매가격,판매일자와판매처,이미지 고유 번호,작품 판매 횟수
0,임상진 Lim SangChin (1935~2013),무제,1983 캔버스에 아크릴,50☓60.6cm,1983.0,600000.0,2020-11-11 에이옥션,380410.0,1.0
1,정술원 Jung SulWon (1885~1959),화조,비단에 수묵담채,25☓24.5cm,NaN,400000.0,2020-11-11 에이옥션,380460.0,1.0
2,정주상 Jeong JuSang (1925~2012),심정흥장 (선면),종이에 먹,56☓17.5cm,NaN,100000.0,2020-11-11 에이옥션,380491.0,1.0
3,이양원 Lee YangWon (1944~),풍속도,종이에 수묵담채,55☓23.5cm,NaN,360000.0,2020-11-11 에이옥션,380417.0,1.0
4,이외수 Lee OiSoo (1946~),사람과 사람들,종이에 채색,25.5☓18.5cm,NaN,240000.0,2020-11-11 에이옥션,380391.0,1.0


## 3. 데이터 전처리

### (1) 작품 판매 횟수가 공백인 경우 1로 처리

In [78]:
data['작품 판매 횟수'][data['작품 판매 횟수'].isna()] = 1

/tmp/ipykernel_20481/3825792433.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['작품 판매 횟수'][data['작품 판매 횟수'].isna()] = 1


### (2) 제작년도의 경우 절반 이상이 공백으로 제거

In [79]:
del data['제작년도']

### (3) 작가 미상 여부 확인 변수 추가

In [80]:
data['작가미상여부'] = 0
data['작가미상여부'][data['작가명'] == '작가미상 Anonymous'] = 1

/tmp/ipykernel_20481/3541825850.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['작가미상여부'][data['작가명'] == '작가미상 Anonymous'] = 1


### (4) 작가 생존 여부 확인 변수 추가

In [81]:
data['작가생존여부'] = '생존'
data['작가생존여부'][data['작가명'] == '작가미상 Anonymous'] = 1

for i in range(len(data['작가생존여부'])):
    painter = data['작가명'][i]
    if painter[-1] != ')':
        data['작가생존여부'][i] = '알수없음'
    elif len(painter.split('(')[-1].split('~')[-1]) > 3:
        data['작가생존여부'][i] = '사망'
        

/tmp/ipykernel_20481/1162102882.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['작가생존여부'][data['작가명'] == '작가미상 Anonymous'] = 1
/tmp/ipykernel_20481/1162102882.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['작가생존여부'][i] = '사망'
/tmp/ipykernel_20481/1162102882.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['작가생존여부'][i] = '알수없음'


### (5) 재료에서 숫자 및 띄어쓰기 제거

##

In [88]:
data['재료'][data['재료'].isna()] = '알수없음'

def ingredient_preprocess(ingredient):
    new_str = re.sub(r"[0-9]", "", str(ingredient))
    new_str = new_str.replace(' ', '')
    if len(new_str) < 1:
        new_str = '알수없음'
    return new_str

# data['재료'] = data['재료'].map(ingredient_preprocess)
for i in range(len(data['재료'])):
    data['재료'][i] = ingredient_preprocess(data['재료'][i])

/tmp/ipykernel_20481/2469359434.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['재료'][data['재료'].isna()] = '알수없음'
/tmp/ipykernel_20481/2469359434.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['재료'][i] = ingredient_preprocess(data['재료'][i])


### (6) 사이즈 가로, 세로 추출

In [127]:
def extract_width(size):
    try:
        if len(re.sub(r"[^0-9, .]", "", size)) < 3:
            return 0
        else:
            if '×' in size:
                width = size.split('×')[0]
                width = re.sub(r"[^0-9, .]", "", width)
            else:
                width = size.split('☓')[0]
                width = re.sub(r"[^0-9, .]", "", width)
        return float(width)
    except:
        return 0

def extract_height(size):
    try:
        if len(re.sub(r"[^0-9, .]", "", size)) < 3:
            return 0
        else:
            if '×' in size:
                height = size.split('×')[1]
                height = re.sub(r"[^0-9, .]", "", height)
            else:
                height = size.split('☓')[1]
                height = re.sub(r"[^0-9, .]", "", height)
        return float(height)
    except:
        return 0

data['가로'] = 0
data['세로'] = 0
data['사이즈'][data['사이즈'].isna()] = '알수없음'

for i in range(len(data['사이즈'])):
    if len(data['사이즈'][i]) > 2 and len(data['사이즈'][i]) < 15:
        data['가로'][i] = extract_height(data['사이즈'][i])
        data['세로'][i] = extract_height(data['사이즈'][i])

/tmp/ipykernel_20481/905040304.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['사이즈'][data['사이즈'].isna()] = '알수없음'
/tmp/ipykernel_20481/905040304.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['가로'][i] = extract_height(data['사이즈'][i])
/tmp/ipykernel_20481/905040304.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['세로'][i] = extract_height(data['사이즈'][i])


### (7) 판매일자 계절 추출

In [149]:
data['판매계절'] = ''

def extract_season(date):
    month = int(date.split('-')[1])
    if month in [12, 1, 2]:
        return '겨울'
    elif month in [3,4,5]:
        return '봄'
    elif month in [6,7,8]:
        return '여름'
    elif month in [9,10,11]:
        return '가을'

data['판매계절'] = data['판매일자와판매처'].map(extract_season)

### (8) 범주형 변수 Onehot encoding

In [162]:
dummies = pd.get_dummies(data.loc[:, ['작가생존여부','판매계절']], drop_first=False)

### (9) 연속형 변수 정규화 (Min-max)

In [159]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
numeric_data =data.loc[:, ['가로', '세로', '작품 판매 횟수']]
numeric_data.loc[:, ['가로', '세로', '작품 판매 횟수']] = min_max_scaler.fit_transform(numeric_data)
numeric_data.head()

,가로,세로,작품 판매 횟수
0,0.065372,0.065372,0.0
1,0.026429,0.026429,0.0
2,0.018878,0.018878,0.0
3,0.025351,0.025351,0.0
4,0.019957,0.019957,0.0


### (10) X, y 정의

In [165]:
X = pd.concat([dummies, numeric_data], axis = 1)
y = data['판매가격']

### (11) Train Test 분할

In [167]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1004)

## 4. Modeling

In [174]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(n_estimators = 500, max_depth = 3, learning_rate = 0.01).fit(X_train, y_train)

In [175]:
test_predict = xgb_model.predict(X_test)

In [176]:
from sklearn.metrics import mean_squared_error, r2_score

print("RMSE: {}".format(np.sqrt(mean_squared_error(y_test, test_predict))))
print("R2 Score: {}".format(r2_score(y_test, test_predict)))

RMSE: 41528953.64799029
R2 Score: -0.03908283296090409
